# CNN from scratch

## Setup


In [16]:
import torch
import torch.nn as nn
from tensorboardX import SummaryWriter
from pa2_sample_code import get_datasets

train_data, eval_data = get_datasets()

# split train set into holdout_train and holdout_eval sets
holdout_train_len = int(len(train_data) * 0.8)
holdout_eval_len = len(train_data) - holdout_train_len
holdout_train_data, holdout_eval_data = torch.utils.data.random_split(train_data, [holdout_train_len, holdout_eval_len])

## Declare model

In [17]:
class CnnClassifier(nn.Module):
    def __init__(self, n_hidden):
        super(CnnClassifier, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=4, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=4, out_channels=8, kernel_size=3, stride=2, padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=2, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=1, padding=0),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=0),
            nn.Sigmoid()
        )
        
        self.predictor = nn.Sequential(
            nn.Linear(in_features=32, out_features=n_hidden),
            nn.ReLU(),
            nn.Linear(in_features=n_hidden, out_features=47)
        )
        
        self.loss_func = nn.CrossEntropyLoss(reduction='sum')
        
    def forward(self, in_data):
        img_features = self.encoder(in_data).view(in_data.size(0), 32)
        logits = self.predictor(img_features)
        return logits

    def loss(self, logits, labels):
        return self.loss_func(logits, labels) / logits.size(0)
    
    def top_k_acc(self, logits, labels, k=1):
        _, k_labels_pred = torch.topk(logits, k=k, dim=1) # shape (n, k)
        k_labels = labels.unsqueeze(dim=1).expand(-1, k) # broadcast from (n) to (n, 1) to (n, k)
        # flatten tensors for comparison
        k_labels_pred_flat = k_labels_pred.reshape(1,-1).squeeze()
        k_labels_flat = k_labels.reshape(1,-1).squeeze()
        # get num_correct
        num_correct = k_labels_pred_flat.eq(k_labels_flat).sum(0)
        return num_correct / labels.size(0)
        

## Training

In [18]:
def train(model, loaders, optimizer, writer, num_epoch=10, device='cpu'):
    # for logging to tensorboard
    loss = {}
    top1 = {}
    top3 = {}
    def run_epoch(mode):
        epoch_loss = 0.0
        epoch_top1 = 0.0
        epoch_top3 = 0.0
        for i, batch in enumerate(loaders[mode], 0):
            in_data, labels = batch
            in_data, labels = in_data.to(device), labels.to(device)

            if mode == 'train':
                optimizer.zero_grad()

            logits = model(in_data)
            batch_loss = model.loss(logits, labels)
            batch_top1 = model.top_k_acc(logits, labels, k=1)
            batch_top3 = model.top_k_acc(logits, labels, k=3)

            epoch_loss += batch_loss.item()
            epoch_top1 += batch_top1
            epoch_top3 += batch_top3

            if mode == 'train':
                batch_loss.backward()
                optimizer.step()

        # sum of all batchs / num of batches
        epoch_loss /= i + 1 
        epoch_top1 /= i + 1
        epoch_top3 /= i + 1
        
        loss[mode] = epoch_loss
        top1[mode] = epoch_top1
        top3[mode] = epoch_top3
        
        print('epoch %d %s loss %.4f top1 %.4f top3 %.4f' % (epoch, mode, epoch_loss, epoch_top1, epoch_top3))
        # log to tensorboard
        if not (writer is None) and (mode == 'eval'):
            writer.add_scalars('%s_loss' % model.__class__.__name__,
                         tag_scalar_dict={'train': loss['train'], 
                                          'eval': loss['eval']}, 
                         global_step=epoch)
            writer.add_scalars('%s_top1' % model.__class__.__name__,
                         tag_scalar_dict={'train': top1['train'], 
                                          'eval': top1['eval']}, 
                         global_step=epoch)
            writer.add_scalars('%s_top3' % model.__class__.__name__,
                         tag_scalar_dict={'train': top3['train'], 
                                          'eval': top3['eval']}, 
                         global_step=epoch)
    for epoch in range(num_epoch):
        run_epoch('train')
        run_epoch('eval')

In [ ]:
for n_hidden in [32, 64]:
    for optimizer_conf in [
        {'optimizer':'adam', 'lr':0.001},
        {'optimizer':'sgd', 'lr':0.1},
        {'optimizer':'sgd', 'lr':0.01},
    ]:
        model = CnnClassifier(n_hidden=n_hidden)
        if optimizer_conf['optimizer'] == 'adam':
            optimizer = torch.optim.Adam(model.parameters(), lr=optimizer_conf['lr'])
        else:
            optimizer = torch.optim.SGD(model.parameters(), lr=optimizer_conf['lr'])
        conf_str = str(n_hidden)+'_'+optimizer_conf['optimizer']+'_'+str(optimizer_conf['lr'])
        print(conf_str)
        train(
            model=model,
            loaders={
                'train': torch.utils.data.DataLoader(holdout_train_data, batch_size=32, shuffle=True),
                'eval': torch.utils.data.DataLoader(holdout_eval_data, batch_size=32)
            },
            optimizer=optimizer, 
            writer=SummaryWriter('./logs/cnn/%s' % (conf_str)), 
            num_epoch=10, 
            device='cpu'
        )

32_adam_0.001
epoch 0 train loss 2.5608 top1 0.0000 top3 0.0000
epoch 0 eval loss 1.6268 top1 0.0000 top3 0.0000
epoch 1 train loss 1.3624 top1 0.0000 top3 0.0000
epoch 1 eval loss 1.1975 top1 0.0000 top3 0.0000
epoch 2 train loss 1.1254 top1 0.0000 top3 0.0000
epoch 2 eval loss 1.1101 top1 0.0000 top3 0.0000
epoch 3 train loss 1.0144 top1 0.0000 top3 0.0000
epoch 3 eval loss 0.9709 top1 0.0000 top3 0.0000
epoch 4 train loss 0.9394 top1 0.0000 top3 0.0000
epoch 4 eval loss 0.9671 top1 0.0000 top3 0.0000
epoch 5 train loss 0.8839 top1 0.0000 top3 0.0000
epoch 5 eval loss 0.8709 top1 0.0000 top3 0.0000
epoch 6 train loss 0.8385 top1 0.0000 top3 0.0000
epoch 6 eval loss 0.8439 top1 0.0000 top3 0.0000
epoch 7 train loss 0.8024 top1 0.0000 top3 0.0000
epoch 7 eval loss 0.8351 top1 0.0000 top3 0.0000
epoch 8 train loss 0.7703 top1 0.0000 top3 0.0000
epoch 8 eval loss 0.8045 top1 0.0000 top3 0.0000
epoch 9 train loss 0.7438 top1 0.0000 top3 0.0000
epoch 9 eval loss 0.7911 top1 0.0000 top3 0.0